<a href="https://colab.research.google.com/github/steveregis/busybox/blob/master/2_Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![DLI Header](images/DLI_Header.png)

# Iterative Prompt Development

In this notebook we warm up by iterating on a set of simple prompts, familiarizing ourselves with the `transformers` pipeline and the LLaMA-2 model we will be using throughout the course.

By iteratively experimenting with seemingly simple prompts, we will begin to see the importance of creating prompts that are **specific**, provide **cues** and will also learn about how to give the model **"time to think"** when given tasks that it might find challenging.

## Learning Objectives

By the time you complete this notebook you will be able to:
- Use a `transformers` pipeline to generate responses from a LLaMA-2 LLM.
- Craft prompts that are **specific**.
- Craft prompts that give the model **"time to think"**.
- Provide **cues** to the model to guide its response.

## Video Walkthrough

Execute the cell below to load the video walkthrough of this notebook.

In [1]:
 from IPython.display import HTML

video_url = "https://d36m44n9vdbmda.cloudfront.net/assets/s-fx-12-v1/v2/02-prompting.mp4"

video_html = f"""
<video controls width="640" height="360">
    <source src="{video_url}" type="video/mp4">
    Your browser does not support the video tag.
</video>
"""

display(HTML(video_html))


In [2]:
!pip install transformers==4.35.0 accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 kB 10.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tokenizers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of tokenizers to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 

In [2]:
!pip install huggingface_hub>=0.14.1

In [5]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 70.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [6]:
!apt-get update && apt-get install -y g++ cmake libssl-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,326 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,648 kB]
Get:14 https://ppa.launch

In [4]:
!pip install auto-gptq
!pip install transformers accelerate


  Using cached auto_gptq-0.7.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached datasets-3.0.1-py3-none-any.whl.metadata (20 kB)
  Using cached rouge-1.0.1-py3-none-any.whl.metadata (4.1 kB)
  Using cached gekko-1.2.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached peft-0.13.2-py3-none-any.whl.metadata (13 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.17-py310-none-any.whl.metadata (7.2 kB)
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 89.8 MB/s eta 0:00:00


In [7]:
!pip install --upgrade huggingface_hub accelerate

In [6]:
from transformers import AutoTokenizer, pipeline
from auto_gptq import AutoGPTQForCausalLM  # Correct GPTQ loader

model_id = "TheBloke/Llama-2-13B-chat-GPTQ"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

# Load the GPTQ model
model = AutoGPTQForCausalLM.from_quantized(model_id, device="cuda:0", use_safetensors=True)

# Create a text-generation pipeline
llama_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

# Test the pipeline
result = llama_pipe("Hello, how are you?")
print(result)


WARNING - Exllamav2 kernel is not installed, reset disable_exllamav2 to True. This may because you installed auto_gptq using a pre-build wheel on Windows, in which exllama_kernels are not compiled. To use exllama_kernels to further speedup inference, you can re-install auto_gptq from source.
WARNING - CUDA kernels for auto_gptq are not installed, this will result in very slow inference speed. This may because:
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads 

quantize_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

INFO - The layer lm_head is not quantized.
INFO:auto_gptq.modeling._base:The layer lm_head is not quantized.
The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'Llam

[{'generated_text': "Hello, how are you?\n\nI'm doing well, thanks for asking! I'm here to help answer any questions you may have. Is there something specific you'd like to know or discuss?"}]


In [10]:
# @title Default title text
from transformers import pipeline
model = "TheBloke/Llama-2-13B-chat-GPTQ"

llama_pipe = pipeline("text-generation", model=model, device_map="auto");

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


PackageNotFoundError: No package metadata was found for auto-gptq

## Create LLaMA-2 Pipeline

## Helper Functions

In this notebook we will use the following function to support our interaction with the LLM.

### Generate Model Responses

In [7]:
def generate(prompt, max_length=1024, pipe=llama_pipe, **kwargs):
    """
    Generates a response to the given prompt using a specified language model pipeline.

    This function takes a prompt and passes it to a language model pipeline, such as LLaMA,
    to generate a text response. The function is designed to allow customization of the
    generation process through various parameters and keyword arguments.

    Parameters:
    - prompt (str): The input text prompt to generate a response for.
    - max_length (int): The maximum length of the generated response. Default is 1024 tokens.
    - pipe (callable): The language model pipeline function used for generation. Default is llama_pipe.
    - **kwargs: Additional keyword arguments that are passed to the pipeline function.

    Returns:
    - str: The generated text response from the model, trimmed of leading and trailing whitespace.

    Example usage:
    ```
    prompt_text = "Explain the theory of relativity."
    response = generate(prompt_text, max_length=512, pipe=my_custom_pipeline, temperature=0.7)
    print(response)
    ```
    """

    def_kwargs = dict(return_full_text=False, return_dict=False)
    response = pipe(prompt.strip(), max_length=max_length, **kwargs, **def_kwargs)
    return response[0]['generated_text'].strip()

## The Capital of California

Let's begin with a very simple prompt, which we will pass to our `generate` function in order to get a response back from the LLaMA-2 model we are using. In this series of prompts we are interested for the model to respond to us with the capital of the state of California, which is *Sacramento*.

Our hope for this experiment is to get back the exact response `"Sacramento"` without anything else in the repsonse.

In [8]:
prompt = "What is the capital of California?"

print(generate(prompt))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


The capital of California is Sacramento.


---

The model did not understand that we only wanted the name of the capital city, without any other context, so let's craft a prompt that is more **specific**.

In [9]:
prompt = "What is the capital of California? Only answer this question and do so in as few a words as possible."

print(generate(prompt))

Sacramento


---

That is an improvement, but we are still getting a leading `Answer: ` in the response. Let's try to prevent this behavior by providing the model with the **cue** `Answer: `. Doing so may prevent the model from providing that text itself.

In [10]:
prompt = "What is the capital of California? Only answer this question and do so in as few a words as possible. Answer: "

print(generate(prompt))

Sacramento.


## Vowels in Sacramento

In the following section we try to get the model to do something a little more complicated: tell us all the vowels in the name of the capital of California.

The correct answer is S**a**cr**a**m**e**nt**o** -> **aaeo** -> **aeo**. It's worth noting that in order to make this easy on myself (and you) I performed multiple steps to arrive at my answer.

In [11]:
prompt = "Tell me the vowels in the capital of California."

print(generate(prompt))

The vowels in the capital of California are "e" and "a".


---

When models are faced with the need to reason in a way that requires multiple steps, it often helps to construct a prompt requesting that the model perform multiple intermediary steps, almost like asking it to show its work. This technique is often referred to as giving the model **"time to think"**.

The prompt below aims for the same end result, but asks the model to take the intermediate step of responding with the capital of California, before then responding with the vowels in it.

In [12]:
prompt = "Tell me the capital of California, and then tell me all the vowels in it."

print(generate(prompt))

The capital of California is Sacramento.

The vowels in Sacramento are A, E, and O.


---

Now that we see the effectiveness of giving the model **"time to think"**, let's try again with a slightly more complicated task: the vowels in the capital of California in reverse alphabetical order.

The correct answer is S**a**cr**a**m**e**nt**o** -> **aaeo** -> **aeo** -> **oea**

In [13]:
prompt = "Tell me the vowels in the capital of California in reverse alphabetical order?"

print(generate(prompt))

I'm thinking... uh... oh, I know this one! The vowels in the capital of California in reverse alphabetical order are... e-a-o-u!


---

In order to assist the model, let's again give it **"time to think"** by prompting it to break down the task into intermediate steps and show its work.

In [14]:
prompt = "Tell me the capital of California, and then tell me all the vowels in it, then tell me the vowels in reverse-alphabetical order."

print(generate(prompt))

I'm not sure what you're asking. The capital of California is Sacramento, and it has the vowels A, E, I, O, and U. In reverse alphabetical order, the vowels would be U, O, I, E, and A. Is there something specific you'd like to know about this?


## Exercise

While LLMs aren't necessarily the best tool for performing math, as an exercise, generate a response from the prompt below, which intends to get the product of multiplying 23 and 34, and then iteratively develop a prompt which results in your getting the correct answer. Be sure to consider how you can be **precise** in your prompt, and also, provide an opportunity for the model to have **"time to think"**.

If you get stuck, a solution is provided below.

In [15]:
23*34 # Show the actual answer

782

In [16]:
prompt = "23x34" # While you and I understand the intention of this prompt, to the model it is not at all **precise**

print(generate(prompt))

cm.

The painting is in good condition. The canvas is tightly stretched, and the paint layers are well-preserved. There are some minor cracks and tiny losses in the paint, but overall the painting is in good condition.

The provenance of the painting is not known, but it is believed to have been created in the early 20th century, possibly in the 1920s or 1930s. The style of the painting is reminiscent of the works of the Russian avant-garde movement, which was active during that time period.

The painting is a unique and interesting piece of art that would make a great addition to any collection. It is a rare and valuable piece of art that is sure to appreciate in value over time.


### Your Work Here

### Solution

Click on the `...` to see a working solution.

In [ ]:
prompt = "Calculate the product of 23 and 34. Use the steps typical of long multiplication and show your work."

print(generate(prompt))

## Key Concept Review

The following key concepts were introduced in this notebook:

- **Precise**: Being as explicit as necessary to guide the response of an LLM.
- **Cue**: A conclusion to a prompt that guides its response, often to prevent it from including the cue itself in its response.
- **"Time to think"**: A quality in prompts that supports LLM responses (often requiring calculation) by asking for the model to take multiple steps and show its work.

## Restart the Kernel

In order to free up GPU memory for the next notebook, please run the following cell to restart the kernel.

In [ ]:
from IPython import get_ipython

get_ipython().kernel.do_shutdown(restart=True)

![DLI Header](images/DLI_Header.png)